In [84]:
import import_ipynb
import GetKeypoints

In [85]:


import numpy as np
import pandas as pd
# import tensorflow as tf
import os,cv2

import warnings
warnings.filterwarnings("ignore")

data_path = './Dataset'



labels=[]
for folder in os.listdir(data_path):
    labels.append(folder)
labels.sort() #len = 107


train_videos=[]
train_labels=[]
train_keypoints = []
import traceback
import random

num = 1

for i,folder in enumerate(labels):
  try:
    for video in os.listdir(data_path+'/'+folder):
      print(num)
      num+=1
      video = os.path.join(data_path+'/'+folder+'/'+video)
      # Open the video file
      cap = cv2.VideoCapture(video)

      # Get the total number of frames in the video
      total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

      # Select a random starting frame number
      start_frame = random.randint(0, total_frames - 16)

      # Set the number of frames to be selected
      num_frames = 16

      # Set the frame number to start with
      cap.set(cv2.CAP_PROP_POS_FRAMES, start_frame)

      # Loop through the frames and save them
      frames = []
      keypoints = []
      for j in range(num_frames):
          ret, image = cap.read()
          if ret:
            frames.append(cv2.resize(image,(112,112)))
            kk = GetKeypoints.getKeyPointsforFrame(image)
            keypoints.append(kk)

      # Release the video file
      cap.release()


      keys = GetKeypoints.CentralisedKeypoints1(keypoints)
      if(keys == None):
            continue
      train_videos.append(frames)
      train_keypoints.append(keys)
      train_labels.append(i)
      # if(num>2):
      #    break

  except Exception:
    traceback.print_exc()

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [5]:
# train_videos = np.asarray(train_videos)
import tensorflow as tf
train_keypoints_array = np.asarray(train_keypoints)
train_labels_array = np.asarray(train_labels).astype('int32')
train_videos_array = np.asarray(train_videos)

from sklearn.model_selection import train_test_split
X_train_videos,X_test_videos,y_train_vidoes,y_test_videos = train_test_split(train_videos_array,train_labels_array,test_size=0.25,shuffle=True,random_state = 42)

from sklearn.model_selection import train_test_split

XKey_train,XKey_test,yKey_train,yKey_test = train_test_split(train_keypoints_array,train_labels_array,test_size=0.25,shuffle=True,random_state = 42)

XKey_train = tf.convert_to_tensor(XKey_train, dtype=tf.float32)
yKey_train = tf.convert_to_tensor(yKey_train, dtype=tf.int32)
XKey_test = tf.convert_to_tensor(XKey_test, dtype=tf.float32)
yKey_test = tf.convert_to_tensor(yKey_test, dtype=tf.int32)

ValueError: setting an array element with a sequence.

In [14]:
# train_videos = np.asarray(train_videos)
new_train_keypoints = np.asarray(train_keypoints)
new_train_labels = np.asarray(train_labels).astype('int32')

# train_keypoints = tf.convert_to_tensor(train_keypoints, dtype=tf.float32)
# # yKey_train = tf.cast(yKey_train , dtype=tf.float32)
# train_labels = tf.convert_to_tensor(train_labels, dtype=tf.float32)

# new_train_keypoints = np.array([np.array(val) for val in new_train_keypoints])

from sklearn.model_selection import train_test_split

XKey_train,XKey_test,yKey_train,yKey_test = train_test_split(new_train_keypoints,new_train_labels,test_size=0.25,shuffle=True,random_state = 42)
XKey_train = tf.convert_to_tensor(XKey_train, dtype=tf.float32)
yKey_train = tf.convert_to_tensor(yKey_train, dtype=tf.int32)

ValueError: setting an array element with a sequence.

In [ ]:
def CalculateAccuracy(videos,keypointss,labels,RGBRatio,PoseRatio):
    count = 0
    for i in range(len(labels)):
        predicted_label = GetKeypoints.predictYogaPose(np.array([videos]),np.array([keypointss]),True,RGBRatio,PoseRatio)
        if predicted_label == labels[i]:
            count+=1

    return count/len(train_labels)

In [ ]:
trainAccuracy = CalculateAccuracy(X_train_videos,X_train_keypoints,y_train_vidoes)
testAccuracy = CalculateAccuracy(X_test_videos,X_test_keypoints,y_test_videos)

In [ ]:
print("train accuracy is:",trainAccuracy)
print("test accuracy is:",testAccuracy)

In [3]:
trainAcc = []
testAcc = []
for i in range(1,10):
    trainAccuracy = CalculateAccuracy(X_train_videos,X_train_keypoints,y_train_vidoes,0.1*i,1-0.1*i)
    testAccuracy = CalculateAccuracy(X_test_videos,X_test_keypoints,y_test_videos,0.1*i,1-0.1*i)
    trainAcc.append(trainAccuracy)
    testAcc.append(testAccuracy)
    

TypeError: 'float' object cannot be interpreted as an integer

In [ ]:
print("ratio","    train Accuracy","    test Accuracy")
for i in range(len(trainAcc)):
    print(i,"      ",trainAcc[i],"       ",testAcc[i])

In [16]:
# # create a DataFrame to store the data
# data1 = pd.DataFrame({'keypoints': train_keypoints, 'label': train_labels})
# # save the DataFrame to a CSV file
# data1.to_csv('train_data_keypoints.csv', index=False)

# data2 = pd.DataFrame({'videos': train_videos, 'label': train_labels})
# # save the DataFrame to a CSV file
# data2.to_csv('train_data_videos.csv', index=False)

In [86]:
np.save('train_data_videos', train_videos)
loaded_array = np.load('train_data_videos.npy')
X = np.array(loaded_array).reshape(loaded_array.shape)
y = np.array(train_labels)

from keras import models
m1 = models.load_model('RGBmodel2.h5')
m1.evaluate(X,y)

38/38 [==============================] - 8s 185ms/step - loss: 38.6364 - accuracy: 0.7766


[38.636356353759766, 0.7765869498252869]

In [87]:
np.save('train_data_keypoints', train_keypoints)
loaded_array = np.load('train_data_keypoints.npy')
X = np.array(loaded_array).reshape(loaded_array.shape)
y = np.array(train_labels)

from keras import models
m1 = models.load_model('PoseStream.h5')
m1.evaluate(X,y)

ValueError: Object arrays cannot be loaded when allow_pickle=False

In [82]:
loaded_array

array([], dtype=float64)

In [83]:
train_keypoints

[]

In [2]:
import numpy as np
import pandas as pd
# import tensorflow as tf
import os,cv2

import warnings
warnings.filterwarnings("ignore")

data_path = '../Dataset'



labels=[]
for folder in os.listdir(data_path):
    labels.append(folder)
labels.sort() #len = 107

for i,folder in enumerate(labels):
    print(i,folder)

0 Adhomukhasvanasana
1 Ardhachakrasana
2 Bhujangasana
3 Dhanurasana
4 Marjariasana
5 Padahastasana
6 Padmasana
7 Pawanmuktasana
8 Phalakasana
9 Sarvangasana
10 Sashankasana
11 Setubandhasana
12 Shavasana
13 Tadasana
14 Trikonasana
15 Ustrasana
16 Vakrasana
17 Virbhadrasana1
18 Virbhadrasana2
19 Vrikshasana
